In [35]:
import yfinance as yf
import pandas as pd

In [40]:
sectors = ['technology', 'healthcare', 'financial-services', 'energy', 'consumer-cyclical']
top_sector_dfs = []
for sector in sectors:
    top_companies = yf.Sector(sector).top_companies
    top_3_companies = top_companies[top_companies['rating'] == "Buy"].head(3)
    top_3_companies['sector'] = sector
    top_sector_dfs.append(top_3_companies)
    
top_companies_df = pd.concat(top_sector_dfs, axis=0)
top_companies_df

,name,rating,market weight,sector
symbol,,,,
NVDA,NVIDIA Corporation,Buy,0.187388,technology
AAPL,Apple Inc.,Buy,0.177521,technology
MSFT,Microsoft Corporation,Buy,0.162559,technology
LLY,Eli Lilly and Company,Buy,0.114839,healthcare
UNH,UnitedHealth Group Incorporated,Buy,0.082445,healthcare
JNJ,Johnson & Johnson,Buy,0.054454,healthcare
BRK-B,Berkshire Hathaway Inc.,Buy,0.100316,financial-services
JPM,JPMorgan Chase & Co.,Buy,0.079909,financial-services
V,Visa Inc.,Buy,0.059984,financial-services
